In [1]:
import numpy as np
import pandas as pd
import os
import sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
import tensorflow as tf
from tensorflow.keras import models

In [2]:
data=pd.read_csv(r"C:\Users\Soham\Desktop\dataset\ipl\madras.csv")


c:\users\soham\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
columns=['match_id','venue','innings','ball','batting_team','bowling_team','striker','bowler','runs_off_bat','extras','wides',
        'noballs','byes','legbyes','penalty']
data=data[columns]

In [4]:
data['total_runs']=data['runs_off_bat']+data['extras']
data=data.drop(columns=['runs_off_bat','extras'])


In [5]:
data=data[data['ball']<6]
data=data[data['innings']<=2]

In [6]:
data=data.groupby(['match_id','venue','innings','batting_team','bowling_team']).total_runs.sum()


In [7]:
data=data.reset_index()


In [8]:
data=data.drop(columns=['match_id'],axis=1)

In [9]:
data=data.replace(to_replace='M.Chinnaswamy Stadium',value='M Chinnaswamy Stadium')
data=data.replace(to_replace='Sardar Patel Stadium, Motera',value='Narendra Modi Stadium')
data.head()

,venue,innings,batting_team,bowling_team,total_runs
0,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,61
1,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,26
2,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,53
3,"Punjab Cricket Association Stadium, Mohali",2,Kings XI Punjab,Chennai Super Kings,63
4,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Daredevils,40


In [10]:
data=data[data['batting_team']!='Deccan Chargers']
data=data[data['batting_team']!='Kochi Tuskers Kerala']
data=data[data['batting_team']!='Pune Warriors']
data=data[data['batting_team']!='Rising Pune Supergiants']
data=data[data['batting_team']!='Gujarat Lions']
data=data[data['batting_team']!='Rising Pune Supergiant']

In [11]:
data=data[data['bowling_team']!='Deccan Chargers']
data=data[data['bowling_team']!='Kochi Tuskers Kerala']
data=data[data['bowling_team']!='Pune Warriors']
data=data[data['bowling_team']!='Rising Pune Supergiants']
data=data[data['bowling_team']!='Gujarat Lions']
data=data[data['bowling_team']!='Rising Pune Supergiant']

In [12]:
data['venue'].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Sawai Mansingh Stadium',
       'MA Chidambaram Stadium, Chepauk', 'Eden Gardens',
       'Dr DY Patil Sports Academy', 'Newlands', "St George's Park",
       'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium', 'Narendra Modi Stadium',
       'Himachal Pradesh Cricket Association Stadium',
       'Subrata Roy Sahara Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
       'Barabati Stadium', 'Maharashtra Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Punjab Cricket Association IS Bindra Stadium, 

In [13]:
data=data.replace(to_replace='Kings XI Punjab',value='Punjab Kings')

data=data.replace(to_replace='Delhi Daredevils',value='Delhi Capitals')

In [14]:
data1=data.groupby('batting_team').total_runs.mean()
data1.head(8)

batting_team
Chennai Super Kings            45.453416
Delhi Capitals                 45.880952
Kolkata Knight Riders          46.532544
Mumbai Indians                 45.666667
Punjab Kings                   46.993939
Rajasthan Royals               44.701389
Royal Challengers Bangalore    44.751479
Sunrisers Hyderabad            48.096491
Name: total_runs, dtype: float64

In [ ]:
data2=data.groupby(['innings','batting_team'])

In [17]:
venue_encoder=LabelEncoder()
team_encoder=LabelEncoder()
data['venue']=venue_encoder.fit_transform(data['venue'])
data['batting_team']=team_encoder.fit_transform(data['batting_team'])
data['bowling_team']=team_encoder.fit_transform(data['bowling_team'])

In [18]:
array=data.to_numpy()

In [19]:
x,y=array[:,:4],array[:,4]

print(x.shape)

(1267, 4)


In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

print(x_train.shape)

(1140, 4)


In [29]:
x_train=tf.convert_to_tensor(x_train,dtype='float32')
y_train=tf.convert_to_tensor(y_train,dtype='float32')

x_test=tf.convert_to_tensor(x_test,dtype='float32')
y_test=tf.convert_to_tensor(y_test,dtype='float32')

y_train=tf.reshape(y_train,[-1,1])
y_test=tf.reshape(y_test,[-1,1])


In [30]:
x_train=tf.keras.utils.normalize(x_train)
#y_train=tf.keras.utils.normalize(y_train)
x_test=tf.keras.utils.normalize(x_test)
#y_test=tf.keras.utils.normalize(y_test)

In [31]:
model=tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(1))
model.compile(loss='MeanSquaredError',optimizer=tf.keras.optimizers.SGD(lr=0.01,momentum=0.9,nesterov=True),metrics='mean_squared_error')
model.fit(x_train,y_train,epochs=500)
model.save('my_model.h5')

Epoch 1/500
36/36 [==============================] - 0s 916us/step - loss: 444.0184 - mean_squared_error: 444.0184
Epoch 2/500
36/36 [==============================] - 0s 970us/step - loss: 155.9473 - mean_squared_error: 155.9473
Epoch 3/500
36/36 [==============================] - 0s 935us/step - loss: 151.0893 - mean_squared_error: 151.0893
Epoch 4/500
36/36 [==============================] - 0s 1ms/step - loss: 148.7852 - mean_squared_error: 148.7852
Epoch 5/500
36/36 [==============================] - 0s 1ms/step - loss: 148.6250 - mean_squared_error: 148.6250
Epoch 6/500
36/36 [==============================] - 0s 946us/step - loss: 148.3298 - mean_squared_error: 148.3298
Epoch 7/500
36/36 [==============================] - 0s 1ms/step - loss: 148.1109 - mean_squared_error: 148.1109
Epoch 8/500
36/36 [==============================] - 0s 988us/step - loss: 147.7766 - mean_squared_error: 147.7766
Epoch 9/500
36/36 [==============================] - 0s 942us/step - loss: 148.4900 - 

In [32]:
model.evaluate(x_test,y_test)

4/4 [==============================] - 0s 1ms/step - loss: 139.6525 - mean_squared_error: 139.6525


[139.6525115966797, 139.6525115966797]

In [33]:
p=model.predict(x_test)


In [34]:
index=1
print(round(p[index][0]))
print(y_test[index])

46
tf.Tensor([39.], shape=(1,), dtype=float32)


In [35]:
joblib.dump(venue_encoder,'venue_encoder.joblib')
joblib.dump(team_encoder,'team_encoder.joblib')

['team_encoder.joblib']

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regressor=RandomForestRegressor()